In [1]:
%matplotlib inline
import cv2
import os
import glob
import subprocess
from matplotlib import pyplot as plt
import numpy as np
import time as t
print ("OpenCV Version : %s " % cv2.__version__)

OpenCV Version : 3.4.0 


In [2]:
#os.chdir('E:/Tools/ffmpeg/ffmpeg-20170112-6596b34-win64-static/bin')
#subprocess.call(['ffmpeg', '-i', 'C:/Temp/imagetemp/fisheries/vids.mp4', '-r', '1/.25', 'C://Temp/imagetemp/fisheries/output/IMG_%03d.jpg'])


In [3]:
ffmpegPath = 'C:/Tools/ffmpeg/bin'
videoInput = 'C:/Temp/imagetemp/fish/Video - LIS3B2-20170817-Lethrinus_laticaudis.mp4'
outputTemp = 'C:/Temp/imagetemp/fish/output'
outputVid = 'C:/Temp/imagetemp/fish/output2.mp4'

In [4]:
fgbg = cv2.createBackgroundSubtractorMOG2()

In [14]:
files = glob.glob(outputTemp + '/*')
for f in files:
    os.remove(f)

cap = cv2.VideoCapture(videoInput)


ret, frame = cap.read()
cou = 0

blobParams = cv2.SimpleBlobDetector_Params()

blobParams.minThreshold = 1;    # the graylevel of images
blobParams.maxThreshold = 2000;

blobParams.filterByColor = False
blobParams.blobColor = 255

# Filter by Area
blobParams.filterByArea = False
blobParams.minArea = 10000

detector = cv2.SimpleBlobDetector_create(blobParams)

while(True):
    cou = cou + 1
    ret, frame = cap.read()  
    
    #cv2.imshow('frame',fgmask)
    if ret==True:
        gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        smoothedFrame = cv2.GaussianBlur(gray_image,(9,9),0)
        
        
        
        diff = cv2.absdiff(smoothedFrame, gray_image)
        
        thresh = 2
        maxValue = 255
 
        # Basic threshold example
        th, dst = cv2.threshold(diff, thresh, maxValue, cv2.THRESH_BINARY);
        
        kernel = np.ones((5,5), np.uint8)

        
        img_dilation = cv2.dilate(th, kernel, iterations=1)
        
        fgmask = fgbg.apply(img_dilation)
        #keypoints = detector.detect(fgmask)
        #im_with_keypoints = cv2.drawKeypoints(frame, keypoints, np.array([]), (255,255,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        # write the flipped frame
        #out.write(frame)
        cv2.imwrite(outputTemp + '/IMG_' + '{0:0{width}}'.format(cou, width=5) + '.jpg', dst)
        
        if cou > 400:
            break
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release everything if job is finished
cap.release()
#out.release()
cv2.destroyAllWindows()
    
   

In [15]:
os.chdir(ffmpegPath)
subprocess.call(['ffmpeg', '-y', '-r', '25', '-start_number', '0', '-i', outputTemp + '/IMG_%05d.jpg', '-vcodec','libx264','-pix_fmt', 'yuv420p', outputVid])

0